In [12]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np
import numpy as np
import os
import pandas as pd
import matplotlib.pylab as plt

from sklearn.utils import shuffle
import pandas as pd
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

META_SRC = "selected/0_0_0_metadata_extended.csv"
TRAIN_DATA_DIR = "selected"
TRAIN_X_LABEL = "uic_value"

VALID_META_SRC = "validation/selected/0_0_0_metadata_extended.csv"
VALID_DATA_DIR = "validation/selected"

train_meta_df = pd.read_csv(META_SRC)
train_meta_df = shuffle(train_meta_df)

valid_meta_df = pd.read_csv(VALID_META_SRC)
valid_meta_df = shuffle(valid_meta_df)


class ImgReader:
    def read(self, file: str, format_="jpg") -> np.ndarray:
        return plt.imread(file, format_)

    def read_multiple(self, files, format_="jpg") -> np.ndarray:
        return np.asarray([self.read(f, format_) for f in files])


img_reader = ImgReader()


def list_files_from_df(root_dir: str, df: pd.DataFrame):
    files = [os.path.join(root_dir, f) for f in df["filename"].values]

    return files


def get_all_images_with_column(data_dir, metadata_df, col_name: str, is_shuffle=False):
    metadata_df.dropna(inplace=True)
    metadata_df = metadata_df[metadata_df['uic_value'].str.len() == 15]
    df = metadata_df[["filename", col_name]]
    if is_shuffle:
        df = shuffle(df)
    files = list_files_from_df(data_dir, df)
    images = img_reader.read_multiple(files)
    col_values = df[col_name].values

    return images, col_values

def normalize_imgs(imgs):
    new_imgs_shape = (imgs.shape[0], imgs.shape[1], imgs.shape[2], 1)
    imgs = imgs.reshape(new_imgs_shape)
    imgs = imgs/255.0
    return imgs

is_shuffle = True
imgs, uids = get_all_images_with_column(TRAIN_DATA_DIR, train_meta_df, TRAIN_X_LABEL, is_shuffle)
imgs = normalize_imgs(imgs)

valid_imgs, valid_types = get_all_images_with_column(VALID_DATA_DIR, valid_meta_df, TRAIN_X_LABEL, is_shuffle)
valid_imgs = normalize_imgs(valid_imgs)


input_shape = (256, 320, 1)
valid_imgs.shape, imgs.shape

((125, 256, 320, 1), (74, 256, 320, 1))

In [47]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np
import numpy as np
import os
import pandas as pd
import matplotlib.pylab as plt

from sklearn.utils import shuffle
import pandas as pd
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

META_SRC = "selected/0_0_0_metadata_extended.csv"
TRAIN_DATA_DIR = "selected"
TRAIN_X_LABEL = "uic_value"

VALID_META_SRC = "validation/selected/0_0_0_metadata_extended.csv"
VALID_DATA_DIR = "validation/selected"

train_meta_df = pd.read_csv(META_SRC)
train_meta_df = shuffle(train_meta_df)

valid_meta_df = pd.read_csv(VALID_META_SRC)
valid_meta_df = shuffle(valid_meta_df)


class ImgReader:
    def read(self, file: str, format_="jpg") -> np.ndarray:
        return plt.imread(file, format_)

    def read_multiple(self, files, format_="jpg") -> np.ndarray:
        return np.asarray([self.read(f, format_) for f in files])


img_reader = ImgReader()


def list_files_from_df(root_dir: str, df: pd.DataFrame):
    files = [os.path.join(root_dir, f) for f in df["filename"].values]

    return files


def get_all_images_with_column(data_dir, metadata_df, col_name: str, is_shuffle=False):
    metadata_df.dropna(inplace=True)
    metadata_df = metadata_df[metadata_df['uic_value'].str.len() == 15]
    df = metadata_df[["filename", col_name]]
    if is_shuffle:
        df = shuffle(df)
    files = list_files_from_df(data_dir, df)
    images = img_reader.read_multiple(files)
    col_values = df[col_name].values

    return images, col_values

def normalize_imgs(imgs):
    new_imgs_shape = (imgs.shape[0], imgs.shape[1], imgs.shape[2], 1)
    imgs = imgs.reshape(new_imgs_shape)
    imgs = imgs/255.0
    return imgs

is_shuffle = True
imgs, uids = get_all_images_with_column(TRAIN_DATA_DIR, train_meta_df, TRAIN_X_LABEL, is_shuffle)
imgs = normalize_imgs(imgs)

valid_imgs, valid_types = get_all_images_with_column(VALID_DATA_DIR, valid_meta_df, TRAIN_X_LABEL, is_shuffle)
valid_imgs = normalize_imgs(valid_imgs)


input_shape = (256, 320, 1)
valid_imgs.shape, imgs.shape

x_img = valid_imgs.reshape(-1, 256 * 320)
x_test = imgs.reshape(-1, 256 * 320)
tables = []
y_test_tables = []
for i in valid_types:
    tables.append([int(i) for i in i.replace('-', '').strip()[:2]])
y = np.asarray(tables)
for i in uids:
    y_test_tables.append([int(i) for i in i.replace('-', '').strip()[:2]])
y_test = np.asarray(y_test_tables)


class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(256 * 320, 256)
        self.bn1 = nn.BatchNorm1d(num_features=256)

        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(num_features=128)

        self.dropout2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(128, 10)
        self.fc4 = nn.Linear(128, 10)
        

    def forward(self, x):
        x = self.dropout(self.bn1(F.relu(self.fc1(x))))
        x = self.dropout2(self.bn2(F.relu(self.fc2(x))))
        x2 = self.fc4(x)
        x = self.fc3(x)
        return F.log_softmax(x), F.log_softmax(x2)

    def name(self):
        return "MLP"


model = MLPNet().to(device)

optimizer = optim.Adamax(model.parameters(), lr=0.007)


def train(epoch=5):
    for i in range(epoch):
        model.train()
        data, target = torch.from_numpy(x_img).type(torch.cuda.FloatTensor), torch.from_numpy(np.asarray(y)).type(torch.cuda.FloatTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        #         print(data.dtype, target.dtype)
        output1, output2 = model(data)
        loss1 = F.nll_loss(output1, torch.tensor(y[:, 0]).type(torch.cuda.LongTensor))
        loss2 = F.nll_loss(output2, torch.tensor(y[:, 1]).type(torch.cuda.LongTensor))
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        print('Train Epoch: \tLoss: {:.6f}'.format(loss.item()))
        
        

        
def test(epoch=1):
    for i in range(epoch):
        model.eval()
        data, target = torch.from_numpy(x_test).float(), torch.from_numpy(np.asarray(y_test)).float()
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        #         print(data.dtype, target.dtype)
        output1, output2 = model(data)
        loss1 = F.nll_loss(output1, torch.tensor(y_test[:, 0]).type(torch.cuda.LongTensor))
        loss2 = F.nll_loss(output2, torch.tensor(y_test[:, 1]).type(torch.cuda.LongTensor))
        loss = loss1 + loss2
        print(f'loss{loss}')


# train()
y[:, 0], y[:, 1]
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()
train()
test()


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 	Loss: 4.752693
Train Epoch: 	Loss: 4.382185
Train Epoch: 	Loss: 4.018901
Train Epoch: 	Loss: 3.743085
Train Epoch: 	Loss: 3.614478
loss6.615778923034668
Train Epoch: 	Loss: 3.489098
Train Epoch: 	Loss: 3.268129
Train Epoch: 	Loss: 3.056163
Train Epoch: 	Loss: 2.935723
Train Epoch: 	Loss: 2.830981
loss4.586660861968994
Train Epoch: 	Loss: 2.710960
Train Epoch: 	Loss: 2.539115
Train Epoch: 	Loss: 2.393106
Train Epoch: 	Loss: 2.242167
Train Epoch: 	Loss: 2.100329
loss3.8824961185455322
Train Epoch: 	Loss: 1.935457
Train Epoch: 	Loss: 1.997348
Train Epoch: 	Loss: 1.743876
Train Epoch: 	Loss: 1.567148
Train Epoch: 	Loss: 1.634711
loss2.468554973602295
Train Epoch: 	Loss: 1.470736
Train Epoch: 	Loss: 1.434754
Train Epoch: 	Loss: 1.189843
Train Epoch: 	Loss: 1.304060
Train Epoch: 	Loss: 1.164233
loss1.7225819826126099
Train Epoch: 	Loss: 1.054721
Train Epoch: 	Loss: 0.957703
Train Epoch: 	Loss: 0.850618
Train Epoch: 	Loss: 0.814303
Train Epoch: 	Loss: 0.782418
loss1.569588303565

## 